In [1]:
import numpy as np
import gurobipy as gbp
import time
import pandas as pd
np.random.seed(352)


In [3]:
n_com=2
n_sup=2
n_dem=3
n_veh=2

In [4]:
#Supply Demand of all commodities
Si=np.array([[[100,200],[200,700]],
             [[300,100],[500,200]],
             [[600,1200],[500,1000]]])
Dj=np.array([[[75,190],[150,320],[200,225]],
             [[120,150],[100,400],[300,250]],
             [[300,250],[400,900],[600,1500]]])
Si.shape,Dj.shape #days X nodes X commodities

((3, 2, 2), (3, 3, 2))

In [5]:
#Vehicle capacity for each vehicle, number of vehicles of type1 at S1, cost of transportation from supply nodes to demand nodes
V_cap=[150,500]
V_num=np.array([[5,7],[2,1]])
V_cost=np.array([[[10,15,20],[18,13,20]],
                 [[40,50,70],[60,30,70]]])

In [13]:
#People Variables
n_pep=np.array([[[25,25],[50,50],[30,30]],
                [[50,50],[70,70],[100,100]],
                [[100,100],[120,120],[200,200]]]) #no of people on each demand node,
mu=[4,10] # consumption rate (comm req per person per day)

In [7]:
n_days=len(Si)
days_range=range(n_days)
days_range

range(0, 3)

In [8]:
# Indices & Variable Names
supply_nodes = n_sup
demand_nodes = n_dem
supply_nodes_range = range(n_sup)
demand_nodes_range = range(n_dem)
comm_range=range(n_com)
all_nodes_len = n_sup*n_dem
ALL_nodes_range = range(all_nodes_len)
print (supply_nodes_range, demand_nodes_range,comm_range, all_nodes_len)

range(0, 2) range(0, 3) range(0, 2) 6


In [9]:
Pc=np.array([Dj[ix]/Dj[ix].sum(axis=0) for ix in range(len(Dj))])
Pc

array([[[0.17647059, 0.2585034 ],
        [0.35294118, 0.43537415],
        [0.47058824, 0.30612245]],

       [[0.23076923, 0.1875    ],
        [0.19230769, 0.5       ],
        [0.57692308, 0.3125    ]],

       [[0.23076923, 0.09433962],
        [0.30769231, 0.33962264],
        [0.46153846, 0.56603774]]])

In [14]:
unserved_people=[[[0,0],[0,0],[0,0]]]
prev_inventory=[Si[0]/4.0]
unserved_people,prev_inventory

([[[0, 0], [0, 0], [0, 0]]],
 [array([[ 25.,  50.],
         [ 50., 175.]])])

In [11]:
dep_time=24
dep_cost=np.exp(1.5031+0.1172*dep_time)-np.exp(1.5031)
dep_cost

70.38538215041021

In [186]:
cols=['Day','S_Node']+['D'+str(i+1)+'_c'+str(j+1) for i in demand_nodes_range for j in comm_range]
cols+=['Total_S_c'+str(i+1) for i in comm_range]
cols+=['D'+str(i+1)+'_v' for i in demand_nodes_range]
cols+=['Total_V']

df=pd.DataFrame(columns=cols,index=None,dtype=int)

cols2=['Day','D_Node']+['Total'+'_c'+str(i+1) for i in comm_range]+['Unserv'+'_c'+str(i+1) for i in comm_range]
df2=pd.DataFrame(columns=cols2,index=None,dtype=int)
df2

unserved_people=[[[0,0],[0,0],[0,0]]]
for day in days_range:
    print ('#'*50,'  Day',day,'  ','#'*50)
    # Create Model, Set MIP Focus, Add Variables, & Update Model
    m = gbp.Model(' -- The Multi Commodity Vehicle Transportation Problem -- ')

    # Set MIP Focus to 2 for optimality
    m.setParam('MIPFocus', 2)
    # m.setParam(gbp.GRB.Param.PoolSearchMode, 1)
    # m.setParam(gbp.GRB.Param.PoolGap, 0.10)

    decision_var = []
    vehicles_var=[]
    unserved_var=[]
    for orig in supply_nodes_range:
        decision_var.append([])
        vehicles_var.append([])
        for dest in demand_nodes_range:
            decision_var[orig].append([])
            vehicles_var[orig].append([])
            for veh in range(n_veh):
                vehicles_var[orig][dest].append(m.addVar(vtype=gbp.GRB.INTEGER,
                                                  name='S'+str(orig+1)+'_D'+str(dest+1)+'_V'+str(veh)))
            for comm in comm_range:
    #             print (comm,decision_var)
                decision_var[orig][dest].append(m.addVar(vtype=gbp.GRB.INTEGER, 
    #                                         obj=Cij[orig][dest],
    #                                            obj=1,
                                            name='S'+str(orig+1)+'_D'+str(dest+1)+'_c'+str(comm+1)))
    for dest in demand_nodes_range:
        unserved_var.append([])
        for comm in comm_range:
            unserved_var[dest].append(m.addVar(vtype=gbp.GRB.INTEGER,
                                              name='D'+str(dest+1)+'_c'+str(comm+1)+'_U'))
    # Update Model Variables
    m.update()       

    first_term=gbp.quicksum(gbp.quicksum((int(Dj[day][dest][comm])-gbp.quicksum(decision_var[orig][dest][comm] for orig in supply_nodes_range))*(Pc[day][dest][comm])
                                for dest in demand_nodes_range) for comm in comm_range)
    second_term=gbp.quicksum(gbp.quicksum(gbp.quicksum(vehicles_var[orig][dest][veh]*V_cost[orig][dest][veh] for veh in range(n_veh)) for dest in demand_nodes_range) for orig in supply_nodes_range)
    third_term=0.1*gbp.quicksum(gbp.quicksum(unserved_var[dest][comm]*dep_cost for comm in comm_range) for dest in demand_nodes_range)
    
    #objective function
    m.setObjective(first_term+second_term+third_term,gbp.GRB.MINIMIZE)

    m.update()

    # Add Supply Constraints
    for orig in supply_nodes_range:
        for comm in comm_range:
            m.addConstr(gbp.quicksum(decision_var[orig][dest][comm]
                                     for dest in demand_nodes_range) - Si[day][orig][comm] - prev_inventory[day][orig][comm] <= 0)
    # Add Demand Constraints
    for dest in demand_nodes_range:  
        for comm in comm_range:
            m.addConstr(gbp.quicksum(decision_var[orig][dest][comm] 
                                     for orig in supply_nodes_range) - Dj[day][dest][comm] <= 0)
    #Add vehicle constraints
    for orig in supply_nodes_range:
        m.addConstr(gbp.quicksum(decision_var[orig][dest][comm]
                                 for dest in demand_nodes_range for comm in comm_range) - gbp.quicksum(V_cap[veh]*V_num[veh][orig] for veh in range(n_veh)) <=0)
    for orig in supply_nodes_range:
        m.addConstr(gbp.quicksum(gbp.quicksum(vehicles_var[orig][dest][veh] for veh in range(n_veh)) for dest in demand_nodes_range) - gbp.quicksum(V_num[veh][orig] for veh in range(n_veh)) <=0)

    #Add commodity constraints
    for orig in supply_nodes_range:
        for dest in demand_nodes_range:
            m.addConstr((-sum(decision_var[orig][dest][comm]
                                for comm in comm_range)/sum(V_cap[veh] for veh in range(n_veh))) + sum(vehicles_var[orig][dest][veh] for veh in range(n_veh))>=0)
            
    for orig in supply_nodes_range:
        for dest in demand_nodes_range:
            m.addConstr((-sum(decision_var[orig][dest][comm]
                                for comm in comm_range)/sum(V_cap[veh] for veh in range(n_veh))) + sum(vehicles_var[orig][dest][veh] for veh in range(n_veh))<=1)

    #Add unserved people contstraints
    for dest in demand_nodes_range:
        for comm in comm_range:
            m.addConstr(unserved_var[dest][comm]<=n_pep[day][dest][comm]+unserved_people[day][dest][comm])

    for dest in demand_nodes_range:
        for comm in comm_range:
            m.addConstr(sum(decision_var[orig][dest][comm] for orig in supply_nodes_range)-
                        ((n_pep[day][dest][comm]+unserved_people[day][dest][comm]-unserved_var[dest][comm])*mu[comm])>=0)
    #  Optimize and Print( Results)
    m.optimize()
    m.write('path.lp')

#     m.display()
    prev_inventory.append([])

    for orig in supply_nodes_range:
        prev_inventory[day+1].append([])
        for comm in comm_range:
            prev_inventory[day+1][orig].append(sum(decision_var[orig][dest][comm].x
                                 for dest in demand_nodes_range))

#     print (prev_inventory)
    unserved_people.append([])
    for dest in demand_nodes_range:
        unserved_people[day+1].append([])
        for comm in comm_range:
            unserved_people[day+1][dest].append(unserved_var[dest][comm].x)

            
    
    #Populate DataFrame
    for ix in supply_nodes_range:
        r_t=[]
        r_t+=[decision_var[ix][iy][iz].x for iy in demand_nodes_range for iz in comm_range]
        r_t+=[Si[day][ix][iy] for iy in comm_range]
        r_t+=[vehicles_var[ix][iy].x for iy in demand_nodes_range]
        r_t+=[V_num[ix]]
        df.loc[len(df)]=[day+1,ix+1]+r_t

    for ix in demand_nodes_range:
        r_t=[]
        r_t+=[n_pep[0][ix][iy] for iy in comm_range]
        r_t+=[unserved_var[ix][iy].x for iy in comm_range]
        df2.loc[len(df2)]=[day+1,int(ix+1)]+r_t
    
    first_term_val=sum(sum((int(Dj[day][dest][comm])-sum(decision_var[orig][dest][comm].x for orig in supply_nodes_range))*(Pc[day][dest][comm])
                                for dest in demand_nodes_range) for comm in comm_range)
    second_term_val=sum(sum(vehicles_var[orig][dest].x*V_cost[orig][dest] for dest in demand_nodes_range) for orig in supply_nodes_range)
    third_term_val=0.1*sum(sum(unserved_var[dest][comm].x*dep_cost for comm in comm_range) for dest in demand_nodes_range)
    
    print ('^'*100)
    print ("First term: ",int(first_term_val)," Second term: ",int(second_term_val)," Third term: ",int(third_term_val))
    print ('^'*100)
    
#     selected = {}
#     Closed = []
#     for v in m.getVars():
#         var = '%s' % v.VarName
#         units=int(v.x)
#         selected[var] = units
#     #     print (var,units)
#         if (var[-1]=='V'):
#             print ('-'*100)
#             print( '| Day:: ',day,' Supply Facility #', var[:2], 'is sending', units, \
#                   'vehicles to Demand Facility #', var[3:5]) 

#         elif var[-2]=='c':
#             print( '| Day:: ',day,' Supply Facility #', var[:2], 'is shipping', units, \
#                                                 'units of commodity',var[-2:], 'to Demand Facility #', var[3:5])
#         elif var[-1]=='U':
#             print ('-'*100)
#             print( '| Day:: ',day,' Demand Facility #', var[:2], 'is not serving', units, \
#                                                 'people with commodity#', var[3:5])
#         else:
#             print ('Hiiiiiiii')

#     print( '******************************************************************************')
#     print( '    | Objective Value --------------------- ', int(m.objVal))
#     print( '    | Supply Facilities ------------------- ', len(Si))
#     print( '    | Total Supply Units ------------------ ', Si.sum())
#     print( '    | Demand Facilites -------------------- ', len(Dj))
#     print( '    | Total Demand Units ------------------ ', Dj.sum())
#     print( '    | Total Potential Combinations -------- ', len(Si)*len(Dj))
#     print( '    | Actual Combinations  ---------------- ', len(selected))
#     # print( '    | Real Time to Optimize (sec.) -------- ', t2)
#     print( '******************************************************************************')
#     print( '  --  The Transportation Simplex with Gurobi --')
#     break

##################################################   Day 0    ##################################################
Changed value of parameter MIPFocus to 2
   Prev: 0  Min: 0  Max: 3  Default: 0
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (mac64)
Optimize a model with 38 rows, 24 columns and 102 nonzeros
Model fingerprint: 0xdeab5c9c
Variable types: 0 continuous, 24 integer (0 binary)
Coefficient statistics:
  Matrix range     [7e-03, 1e+01]
  Objective range  [2e-01, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+03]
Found heuristic solution: objective 1895.7000680
Presolve removed 7 rows and 0 columns
Presolve time: 0.00s
Presolved: 31 rows, 24 columns, 93 nonzeros
Variable types: 0 continuous, 24 integer (0 binary)
Presolve removed 14 rows and 6 columns
Presolved: 17 rows, 18 columns, 51 nonzeros


Root relaxation: objective 4.810707e+02, 11 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj

In [187]:
df

,Day,S_Node,D1_c1,D1_c2,D2_c1,D2_c2,D3_c1,D3_c2,Total_S_c1,Total_S_c2,D1_v,D2_v,D3_v,Total_V
0,1.0,1.0,75.0,190.0,0.0,0.0,0.0,0.0,100.0,200.0,2.0,0.0,0.0,5.0
1,1.0,2.0,0.0,0.0,150.0,320.0,200.0,225.0,200.0,700.0,0.0,4.0,3.0,7.0
2,2.0,1.0,120.0,150.0,0.0,0.0,100.0,0.0,300.0,100.0,2.0,0.0,1.0,5.0
3,2.0,2.0,0.0,0.0,100.0,400.0,200.0,250.0,500.0,200.0,0.0,4.0,3.0,7.0
4,3.0,1.0,110.0,40.0,-0.0,-0.0,120.0,480.0,600.0,1200.0,1.0,0.0,4.0,5.0
5,3.0,2.0,-0.0,-0.0,348.0,702.0,-0.0,-0.0,500.0,1000.0,0.0,7.0,0.0,7.0


In [156]:
first_term

<gurobi.LinExpr: 417.60704281712685 + -0.17647058823529413 S1_D1_c1 + -0.17647058823529413 S2_D1_c1 + -0.35294117647058826 S1_D2_c1 + -0.35294117647058826 S2_D2_c1 + -0.47058823529411764 S1_D3_c1 + -0.47058823529411764 S2_D3_c1 + -0.2585034013605442 S1_D1_c2 + -0.2585034013605442 S2_D1_c2 + -0.43537414965986393 S1_D2_c2 + -0.43537414965986393 S2_D2_c2 + -0.30612244897959184 S1_D3_c2 + -0.30612244897959184 S2_D3_c2>

In [149]:
df2

,Day,D_Node,Total_c1,Total_c2,Unserv_c1,Unserv_c2
0,1.0,1.0,25.0,25.0,7.0,6.0
1,1.0,2.0,50.0,50.0,13.0,18.0
2,1.0,3.0,30.0,30.0,-0.0,8.0
3,2.0,1.0,25.0,25.0,2.0,16.0
4,2.0,2.0,50.0,50.0,38.0,28.0
5,2.0,3.0,30.0,30.0,0.0,13.0
6,3.0,1.0,25.0,25.0,-0.0,22.0
7,3.0,2.0,50.0,50.0,1.0,8.0
8,3.0,3.0,30.0,30.0,-0.0,10.0
